<a href="https://colab.research.google.com/github/NBELab/NeMo/blob/main/tutorials/image_input.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nengo

In [3]:
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import nengo
from nengo.dists import Uniform

In [ ]:
#input images
h = 25
w = 30
image = np.random.randint(0, 256, (h, w), dtype=np.uint8)

plt.imshow(image, cmap='gray')


In [ ]:

gray_input = 1*np.reshape(image/255.0, (1,1,h*w))
print(gray_input.shape)


In [43]:
present_time = 1
n = h*w

net = nengo.Network(label="Image_input")
with net:
  gray_node = nengo.Node(nengo.processes.PresentInput(
      gray_input, presentation_time=present_time
  ))

  ens_arr1 = nengo.networks.EnsembleArray(
        1,
        h*w,
        intercepts=Uniform(0, 0),
        max_rates=Uniform(200, 200),
        encoders=[[1]],
        neuron_type=nengo.SpikingRectifiedLinear()
    )
  ens_arr2 = nengo.networks.EnsembleArray(
        5,
        h*w,
        max_rates=Uniform(100, 200),
        neuron_type=nengo.LIF()
    )
  nengo.Connection(gray_node, ens_arr1.input)
  nengo.Connection(gray_node, ens_arr2.input,transform=1)

  out1 = nengo.Probe(ens_arr1.output, synapse=0.05)
  out2 = nengo.Probe(ens_arr2.output, synapse=0.05)

print(net.n_neurons)

4500


In [ ]:
import nengo_ocl
runtime = 1 * present_time
with nengo.Simulator(net) as sim:
#with nengo.Simulator(nengo_converter.net) as sim:
    sim.run(runtime)

In [ ]:
image_out1 = np.reshape(sim.data[out1][-1],(h,w))
image_out2 = np.reshape(sim.data[out2][-1],(h,w))


plt.figure()
plt.imshow(image_out1, cmap = plt.get_cmap('gray'))
plt.title('1 SpikingRectifiedLinear')
plt.figure()
plt.imshow(image_out2, cmap = plt.get_cmap('gray'))
plt.title('5 LIF')
plt.figure()
plt.imshow(image, cmap = plt.get_cmap('gray'))
plt.title('original')

In [ ]:
rms_error1 = np.sqrt(np.mean((image_out1 - image/255.0)**2))
rms_error2 = np.sqrt(np.mean((image_out2 - image/255.0)**2))

print("SpikingRectifiedLinear RMSE: ",rms_error1)
print("LIF RMSE: ", rms_error2)